#### Ex01-2 Minimizing free time of resources with smallest no. of active resource (work balance)

In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from datetime import datetime, timedelta

In [2]:
df_manpowers = pd.read_excel("scheduling.xlsx", sheet_name="Manpowers")
df_customer_orders = pd.read_excel("scheduling.xlsx", sheet_name="Customer_mockup")

df_customer_orders["Schedule Delivery"] = df_customer_orders["Schedule Delivery"].astype(str)

manpower_list = df_manpowers["Name"].to_list()
order_list = df_customer_orders["Order"].to_list()

max_manpower_possible = len(df_manpowers)
regular_work_mins = 8 * 60

In [3]:
# Define calendar
start_date = datetime.strptime(
    df_customer_orders["Schedule Delivery"].min().split(" ")[0], "%Y-%m-%d"
)
end_date = datetime.strptime(
    df_customer_orders["Schedule Delivery"].max().split(" ")[0], "%Y-%m-%d"
)

date_modified = start_date
calendar_list = [start_date.strftime("%Y-%m-%d")]

while date_modified < end_date:
    date_modified += timedelta(days=1)
    calendar_list.append(date_modified.strftime("%Y-%m-%d"))

In [4]:
def get_object_from_attr(df, attr):
    object_attr = {}
    for date in calendar_list:
        for order in order_list:
            try:
                object_attr[(date, order)] = df[attr][
                    (df["Order"] == order)
                    & (df["Schedule Delivery"] == date)
                    ].item()
            except ValueError:
                object_attr[(date, order)] = 0
    return object_attr

In [5]:
priority_scores = get_object_from_attr(df_customer_orders, "Priority Score")
amounts = get_object_from_attr(df_customer_orders, "Amount")
cycle_times = get_object_from_attr(df_customer_orders, "Cycle Time (Min)")
capacity_manpowers = get_object_from_attr(df_customer_orders, "Manpower Capacity")
delivery_dates = get_object_from_attr(df_customer_orders, "Schedule Delivery")
daily_requirements = {}
for key in cycle_times:
    daily_requirements[key] = cycle_times[key] * amounts[key]

In [6]:
# daily_requirements = cycle_times * amounts
daily_requirements = {}
for day in calendar_list:
    for order in order_list:
        try:
            daily_requirements[(day, order)] = df_customer_orders["Cycle Time (Min)"][
                (df_customer_orders["Order"] == order)
                & (df_customer_orders["Schedule Delivery"] == day)
                ].item() * df_customer_orders["Amount"][
                (df_customer_orders["Order"] == order)
                & (df_customer_orders["Schedule Delivery"] == day)
                ].item()
        except ValueError:
            daily_requirements[(day, order)] = 0

In [7]:
model = gp.Model('Scheduling')

# Manpower Working Status in each date of each order
gp_manpower_working = model.addVars(calendar_list, manpower_list, order_list, vtype=gp.GRB.BINARY, name="Manpower_working")
# Number of manpowers should be less than or equal to the capacity
model.addConstrs(gp_manpower_working.sum(date, "*", order) <= capacity_manpowers[(date, order)] for date in calendar_list for order in order_list)

# Requirement Time for each order given the manpower and date
gp_requirement_time = model.addVars(calendar_list, manpower_list, order_list, lb=0, ub=regular_work_mins, vtype=gp.GRB.CONTINUOUS, name="Requirement_time")
# Total requirement time done by all manpowers should be equal to the daily requirements
model.addConstrs(gp_requirement_time.sum(date, "*", order) == daily_requirements[(date, order)]
                    for date in calendar_list for order in order_list)

# Work Time for each order given the manpower and date
gp_worktime = model.addVars(calendar_list, manpower_list, order_list, lb=0, ub=regular_work_mins, vtype=gp.GRB.CONTINUOUS, name="Worktime")
# Work Time for each manpower should be equal to the requirement time but considered with manpower status
model.addConstrs(gp_worktime[(date, manpower, order)] == gp_requirement_time[(date, manpower, order)] * gp_manpower_working[(date, manpower, order)]
                    for date in calendar_list for manpower in manpower_list for order in order_list)
# Total work time of all orders for each of manpower in each day should not exceed the regular work time
model.addConstrs(gp_worktime.sum(date, manpower, "*") <= regular_work_mins for date in calendar_list for manpower in manpower_list)

# Free Time for each manpower in each day
gp_freetime = model.addVars(calendar_list, manpower_list, lb=0, ub=regular_work_mins, vtype=gp.GRB.CONTINUOUS, name="Availabletime")
# Free Time for each manpower should be equal to the regular work time minus the work time
model.addConstrs(gp_freetime[(date, manpower)] == regular_work_mins - gp_worktime.sum(date, manpower, "*") for date in calendar_list for manpower in manpower_list)

# Work Balance Constraints => Work time for each manpower should be equal to the daily requirement / number of active manpowers
model.addConstrs(gp_worktime[(date, manpower, order)] * gp_manpower_working.sum(date, "*", order) <= daily_requirements[(date, order)]
                    for date in calendar_list for manpower in manpower_list for order in order_list)

# Minimizing free time of resources
model.ModelSense = gp.GRB.MINIMIZE

objective = 0
objective += gp.quicksum(gp_freetime[(date, manpower)] for date in calendar_list for manpower in manpower_list)

model.setObjective(objective)
model.optimize()

sol = pd.DataFrame(data={"Solution": model.X}, index=model.VarName)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-03
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 144 rows, 888 columns and 1176 nonzeros
Model fingerprint: 0x8ea5ecea
Model has 576 quadratic constraints
Variable types: 600 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+02]
  RHS range        [4e+00, 6e+02]
  QRHS range       [6e+02, 6e+02]
Presolve removed 130 rows and 816 columns
Presolve time: 0.07s
Presolved: 278 rows, 240 columns, 864 nonzeros
Variable types: 216 continuous, 24 integer (24 binary)
Found heuristic solution: objective 6720.0000000

Root relaxation: objective 4.320000e+03, 103 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current N

In [8]:
rows = calendar_list.copy()
columns = manpower_list.copy()
worktime_obj = {}

manpower_plan_by_order = pd.DataFrame(columns=columns, index=rows, data="")

for year, mine, order in gp_worktime.keys():
    try:
        worktime_obj[order].loc[year, mine] = np.round(gp_worktime[year, mine, order].x, 2)
        if worktime_obj[order].loc[year, mine] > 0:
            manpower_plan_by_order.loc[year, mine] = manpower_plan_by_order.loc[year, mine] + f" {order}={np.round(gp_worktime[year, mine, order].x, 2)}"
    except:
        worktime_obj[order] = pd.DataFrame(columns=columns, index=rows, data=0.0)
        worktime_obj[order].loc[year, mine] = np.round(gp_worktime[year, mine, order].x, 2)
        if worktime_obj[order].loc[year, mine] > 0:
            manpower_plan_by_order.loc[year, mine] = manpower_plan_by_order.loc[year, mine] + f" {order}={np.round(gp_worktime[year, mine, order].x, 2)}"
    
manpower_plan_by_order

,Mr. A,Mr. B,Mr. C,Mr. D,Mr. E,Mr. F
2021-11-01,A01=300.0,A02=300.0,,A02=300.0,,A01=300.0
2021-11-02,B03=150.0 B04=300.0,B01=300.0 B03=150.0,B02=300.0 B03=150.0,B02=300.0,B03=150.0 B04=300.0,B01=300.0
2021-11-03,B06=300.0 B07=150.0,B07=150.0,B06=300.0 B07=150.0,B05=200.0,B05=200.0,B05=200.0 B07=150.0
2021-11-04,B09=300.0 B10=150.0,B10=150.0,B09=300.0 B10=150.0,B08=200.0,B08=200.0,B08=200.0 B10=150.0
